# 신경망 모델 구성하기

**신경망은 데이터에 대한 연산을 수행하는 계층(layer) / 모듈(module)로 구성**import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms되어 있다. <br>
torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공한다. <br>
PyTorch의 모든 모듈은 nn.Module 의 하위 클래스(subclass)이다. <br>
신경망은 다른 모듈(계층; layer)로 구성된 모듈이다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있다. 

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 학습을 위한 장치 얻기 

가능한 경우 GPU 또는 MPS와 같은 하드웨어 가속기에서 모델을 학습하려고 한다. <br>
torch.cuda 또는 torch.backends.mps가 사용 가능한지 확인하고, 불가능한 경우 CPU를 계속 사용한다.

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


## 클래스 정의하기

신경망 모델을 nn.Module의 하위클래스로 정의하고, \__init__ 에서 신경망 계층들을 초기화한다. <br>
nn.Module을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현한다.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

NeuralNetwork의 인스턴스(instance)를 생성하고 이를 device로 이동한 뒤, 구조(structure)를 출력한다.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델을 사용하기 위해 입력 데이터를 전달한다. <br>
이는 일부 백그라운드 연산들 과 함께 모델의 forward를 실행한다. model.forward() 직접 호출 X <br>

모델에 입력을 전달하여 호출하면 2차원 텐서를 반환한다. <br>
2차원 텐서의 dim=0은 각 분류(class)에 대한 원시(raw) 예측값 10개가, dim=1에는 각 출력의 개별 값들이 해당한다. <br>
원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻다.

In [5]:
X = torch.rand(1, 28, 28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim = 1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class : {y_pred}")

Predicted class : tensor([6])


## 모델 계층(Layer)

In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

nn.Flatten 계층을 초기화하여 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환한다. (dim=0의 미니배치 차원은 유지)

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

선형 계층 은 저장된 가중치(weight)와 편향(bias)을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈이다. 

In [8]:
layer1 = nn.Linear(in_features = 28 * 28, out_features = 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한 관계(mapping)를 만든다. <br>
비선형 활성화는 선형 변환 후에 적용되어 비선형성(nonlinearity) 을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕는다. <br>
이 모델에서는 nn.ReLU 를 선형 계층들 사이에 사용하지만, 모델을 만들 때는 비선형성을 가진 다른 활성화를 도입할 수도 있다.

In [10]:
print(f"Before ReLU : {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU : tensor([[0.0550, 0.0000, 0.0000, 0.2826, 0.0000, 0.0000, 0.0000, 0.6215, 0.0000,
         0.0000, 0.1061, 0.4159, 0.0729, 0.0000, 0.0000, 0.0000, 0.2014, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0802, 0.7210, 0.4435, 0.0000, 0.0000, 0.3338, 0.0000,
         0.0000, 0.0000, 0.4058, 0.2215, 0.0000, 0.0189, 0.0000, 0.2231, 0.0000,
         0.0845, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.2286, 0.5757, 0.0000, 0.0000, 0.1504, 0.0000,
         0.0000, 0.0941, 0.2163, 0.0813, 0.0000, 0.0472, 0.0000, 0.0000, 0.0000,
         0.1785, 0.0000]], grad_fn=<ReluBackward0>)


After ReLU: tensor([[0.0550, 0.0000, 0.0000, 0.2826, 0.0000, 0.0000, 0.0000, 0.6215, 0.0000,
         0.0000, 0.1061, 0.4159, 0.0729, 0.0000, 0.0000, 0.0000, 0.2014, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0802, 0.7210, 0.4435, 0.0000, 0.0000, 0.3338, 0.0000,
         0.0000, 0.0000, 0.4058, 0.2215, 0.0000, 0.0189, 0.0000, 0.2231, 0.0000,
         0.0845, 0.0000],
      

### nn.Sequential

nn.Sequential은 순서를 갖는 모듈의 컨테이너이다. <br>
데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달된다. <br> 
순차 컨테이너(sequential container)를 사용하여 아래의 seq_modules 와 같은 신경망을 빠르게 만들 수 있다.

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

### nn.Softmax

신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 ([-infty, infty] 범위의 원시 값(raw value)인) logits를 반환한다. <br>
logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정(scale)된다. <br>
dim 매개변수는 값의 합이 1이 되는 차원을 나타낸다.

In [12]:
softmax = nn.Softmax(dim = 1)
pred_probab = softmax(logits)

## 모델 매개변수

신경망 내부의 많은 계층들은 매개변수화(parameterize)된다. <br>
즉, 학습 중에 최적화되는 가중치와 편향과 연관지어진다. nn.Module 을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며, 모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게 된다. <br>

이 예제에서는 각 매개변수들을 순회하며(iterate), 매개변수의 크기와 값을 출력한다.

In [13]:
print(f"Model structure : {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer : {name} | Size : {param.size()} | Values : {param[:2]} \n")

Model structure : NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer : linear_relu_stack.0.weight | Size : torch.Size([512, 784]) | Values : tensor([[-0.0006, -0.0079,  0.0064,  ..., -0.0100, -0.0267, -0.0105],
        [-0.0140, -0.0158,  0.0343,  ..., -0.0172,  0.0031,  0.0122]],
       grad_fn=<SliceBackward0>) 

Layer : linear_relu_stack.0.bias | Size : torch.Size([512]) | Values : tensor([0.0234, 0.0068], grad_fn=<SliceBackward0>) 

Layer : linear_relu_stack.2.weight | Size : torch.Size([512, 512]) | Values : tensor([[-0.0269,  0.0358, -0.0428,  ..., -0.0435, -0.0021, -0.0383],
        [ 0.0027,  0.0416, -0.0154,  ...,  0.0260,  0.0371, -0.0120]],
       grad_fn=<SliceBackward0>) 

Layer : linear_relu_stack.2.b